In [1]:
import random
import itertools
import simpy
import numpy as np

In [2]:
class EmergencyDepartment:
    def __init__(self, env, params):
        self.env = env
        self.params = params
        self.patient_records = []
        
        # all components
        self.reception = simpy.Resource(env, capacity=params['reception_capacity'])
        self.triage = simpy.Resource(env, capacity=params['triage_capacity'])
        self.fast_track_treatment = simpy.Resource(env, capacity=params['fast_track_treatment_capacity'])
        self.main_ed_treatment = simpy.Resource(env, capacity=params['main_ed_treatment_capacity'])
        self.lab = simpy.Resource(env, capacity=params['lab_capacity'])
        self.disposition = simpy.Resource(env, capacity=params['disposition_capacity'])
        self.discharge_fast_track = simpy.Resource(env, capacity=params['discharge_fast_track_capacity'])
        self.discharge_main_ed = simpy.Resource(env, capacity=params['discharge_main_ed_capacity'])
        self.transfer = simpy.Resource(env, capacity=params['transfer_capacity'])

    def patient_arrival(self):
        # this is to allow for numbering the patients
        patient_count = itertools.count()
        
        while True:
            # use a poisson distribution to set up patient flow
            current_hour = int(self.env.now / 60) % 24
            current_rate = self.params['arrival_rates'][current_hour]
            interarrival_time = np.random.exponential(1.0 / (current_rate / 60))
            yield self.env.timeout(interarrival_time)
            self.env.process(self.patient_flow(next(patient_count)+1))

    def patient_flow(self, name):
        # time records of each component for each patient
        patient_time_record = {
            'reception': {'process_time': 0, 'waiting_time': 0},
            'triage': {'process_time': 0, 'waiting_time': 0},
            'moving': 0,
            'fast_track_treatment': {'process_time': 0, 'waiting_time': 0},
            'main_ed_treatment': {'process_time': 0, 'waiting_time': 0},
            'lab': {'process_time': 0, 'waiting_time': 0},
            'disposition': {'process_time': 0, 'waiting_time': 0},
            'discharge_fast_track': {'process_time': 0, 'waiting_time': 0},
            'discharge_main_ed': {'process_time': 0, 'waiting_time': 0},
            'transfer': {'process_time': 0, 'waiting_time': 0},
        }

        # Reception
        yield self.env.process(self.use_resource_with_wait(self.reception, 'reception_time', 'reception', patient_time_record))

        # Triage
        yield self.env.process(self.use_resource_with_wait(self.triage, 'triage_time', 'triage', patient_time_record))
        
        urgency = "Less Urgent" if random.random() < self.params['fast_track_probability'] else "More Urgent"

        if urgency == "Less Urgent":
            # Fast Track
            yield self.env.process(self.use_resource_with_wait(self.fast_track_treatment, 'fast_track_treatment_time', 'fast_track_treatment', patient_time_record))
            
            # Discharge from Fast Track
            yield self.env.process(self.use_resource_with_wait(self.discharge_fast_track, 'discharge_fast_track_time', 'discharge_fast_track', patient_time_record))
        
        else:
            # Main ED
            yield self.env.process(self.use_resource_with_wait(self.main_ed_treatment, 'main_ed_treatment_time', 'main_ed_treatment', patient_time_record))

            # Lab Tests      
            if random.random() < self.params['lab_test_probability']:
                yield self.env.process(self.use_resource_with_wait(self.lab, 'lab_time', 'lab', patient_time_record))
                
                additional_treatment_time = random.uniform(*self.params['main_ed_treatment_time'])
                yield self.env.timeout(additional_treatment_time)
                patient_time_record['main_ed_treatment']['process_time'] += additional_treatment_time

            # Disposition Decision
            yield self.env.process(self.use_resource_with_wait(self.disposition, 'disposition_time', 'disposition', patient_time_record))

            if random.random() < self.params['discharge_probability']:
                yield self.env.process(self.use_resource_with_wait(self.discharge_main_ed, 'discharge_main_ed_time', 'discharge_main_ed', patient_time_record))
            else:
                yield self.env.process(self.use_resource_with_wait(self.transfer, 'transfer_time', 'transfer', patient_time_record))

        self.patient_records.append(patient_time_record)

    def use_resource_with_wait(self, resource, time_param, resource_name, patient_time_record):
        # patient moves to the component first
        moving_time = random.uniform(*self.params['moving_time'])
        yield self.env.timeout(moving_time)
        patient_time_record['moving'] += moving_time

        start_wait = self.env.now
        with resource.request() as req:

            # waiting time before patient actually starts the component
            yield req
            wait_time = self.env.now - start_wait
            patient_time_record[resource_name]['waiting_time'] += wait_time
            
            process_time = random.uniform(*self.params[time_param])
            yield self.env.timeout(process_time)
            patient_time_record[resource_name]['process_time'] += process_time

In [3]:
# Parameters
params = {
    'arrival_rates': [ # across the day
        2, 1, 1, 1, 1, 2,  # 12 AM to 6 AM
        3, 4, 5, 6, 7, 8,  # 6 AM to 12 PM
        9, 9, 8, 7, 8, 9,  # 12 PM to 6 PM
        8, 7, 6, 5, 4, 3   # 6 PM to 12 AM
    ],
    'fast_track_probability': 0.3,
    'lab_test_probability': 0.5,
    'discharge_probability': 0.7,
    
    # component capacities
    'reception_capacity': 1,
    'triage_capacity': 2,
    'fast_track_rooms_capacity': 3,
    'main_ed_rooms_capacity': 5,
    'fast_track_treatment_capacity': 2,
    'main_ed_treatment_capacity': 4,
    'lab_capacity': 2,
    'disposition_capacity': 2,
    'discharge_fast_track_capacity': 1,
    'discharge_main_ed_capacity': 2,
    'transfer_capacity': 1,
    
    # time ranges for each process (min, max) in minutes
    'moving_time': (2, 5),
    'reception_time': (2, 5),
    'triage_time': (3, 8),
    'fast_track_room_time': (10, 20),
    'fast_track_treatment_time': (20, 120),
    'main_ed_room_time': (15, 30),
    'main_ed_treatment_time': (30, 60),
    'lab_time': (30, 60),
    'disposition_time': (1, 3),
    'discharge_fast_track_time': (5, 10),
    'discharge_main_ed_time': (10, 20),
    'transfer_time': (20, 40),
}


env = simpy.Environment()
ed = EmergencyDepartment(env, params)
env.process(ed.patient_arrival())
env.run(until=1440)  # 24 hours (1440 minutes)

In [4]:
mean_metrics = {
    'reception': {'process_time': [], 'waiting_time': []},
    'triage': {'process_time': [], 'waiting_time': []},
    'fast_track_treatment': {'process_time': [], 'waiting_time': []},
    'main_ed_treatment': {'process_time': [], 'waiting_time': []},
    'lab': {'process_time': [], 'waiting_time': []},
    'disposition': {'process_time': [], 'waiting_time': []},
    'discharge_fast_track': {'process_time': [], 'waiting_time': []},
    'discharge_main_ed': {'process_time': [], 'waiting_time': []},
    'transfer': {'process_time': [], 'waiting_time': []},
}


for component in mean_metrics:
    for time in mean_metrics[component]:
        for i in range(len(ed.patient_records)):
            # if process_time==0, it means patient didn't go to that component in the first place
            if ed.patient_records[i][component]['process_time'] > 0:
                mean_metrics[component][time].append(ed.patient_records[i][component][time])

for component in mean_metrics:
    for time in mean_metrics[component]:
        mean_metrics[component][time] = np.array(mean_metrics[component][time]).mean()

mean_metrics

{'reception': {'process_time': 3.4950529662949545,
  'waiting_time': 0.8734727982771897},
 'triage': {'process_time': 5.505255188473252,
  'waiting_time': 0.07356053880894012},
 'fast_track_treatment': {'process_time': 71.97447251111468,
  'waiting_time': 45.948661193100406},
 'main_ed_treatment': {'process_time': 62.61849006760318,
  'waiting_time': 16.961853419897363},
 'lab': {'process_time': 46.70105225622713,
  'waiting_time': 16.539591918209915},
 'disposition': {'process_time': 2.09210541232148, 'waiting_time': 0.0},
 'discharge_fast_track': {'process_time': 7.533198515713447,
  'waiting_time': 0.5736758143170629},
 'discharge_main_ed': {'process_time': 15.701228233939734,
  'waiting_time': 0.2895042262718854},
 'transfer': {'process_time': 29.889903430200807,
  'waiting_time': 19.411359502794998}}